# Домашнее задание
Откуда берутся датасеты

### Цель:
В этом ДЗ вы напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.


### Описание/Пошаговая инструкция выполнения домашнего задания:
По аналогии с занятием по парсингу данных, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).

Идеальный датасет должен иметь текстовое описание некоторого объекта и некоторую целевую переменную, соответствующую этому объекту. Например:
- Сайт новостей: текстовое описание - сама новость, целевая переменная - количество просмотров новости (можно поделить на число дней с момента даты публикации, чтобы получить “среднее число просмотров в день”).
- Сайт с товарами/книгами/фильмами: текстовое описание товара/книги/фильма + средний рейтинг в качестве целевой переменной.
- Блоги - тексты заметок + число просмотров.
- И любые другие ваши идеи, которые подходят под такой формат.

Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.
Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла)

При необходимости очистить датасет от мусора с помощью регулярных выражений).

Посчитать статистики по собранным данным и провести EDA собранных данных (в случае, если данные представляют собой текст - посчитать частотности слов, выявить наиболее частотные слова и т. п)

Не забудьте сохранить полученный датасет, он вам еще пригодиться в дальнейших домашних заданиях.

Критерии оценки:
- Написан парсер, но датасет по каким-то причинам получить не удалось (например, из-за блокировок) - 4 балла
- Написан парсер и собран датасет - 8 баллов
- Проведен EDA собранных данных - 2 балла

In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

In [107]:
def get_hrefs_from_page(page_number):
  page_url = f'https://shazoo.ru/tags/590/anime?page={page_number}'
  response = requests.get(page_url, headers={'User-Agent': UserAgent().chrome})

  if not response.ok:
    return []

  html = response.content
  soup = BeautifulSoup(html, 'html.parser')
  obj = soup.findAll('h4', attrs={'class': 'leading-normal'})

  hrefs_list = []

  for i, el in enumerate(obj):
    hrefs_list.append(el.a['href'])

  return hrefs_list

In [132]:
def get_all_data_from_post(post_url):
  response = requests.get(post_url, headers={'User-Agent': UserAgent().chrome})

  html = response.content
  soup = BeautifulSoup(html, 'html.parser')

  title_obj = soup.find('h1', attrs={'class': 'sm:max-w-4xl text-xl sm:text-3xl leading-tight font-bold break-words dark:text-gray-300'})
  title = title_obj.text


  tags_list = []
  tags_obj = soup.findAll('ul', attrs={'class': 'flex flex-wrap gap-x-2 gap-y-1'})
  tags_li_soup = BeautifulSoup(str(tags_obj), 'html.parser')
  tags_li_list = tags_li_soup.findAll('li' )

  for el in tags_li_list:
    tags_list.append(el.a.text)

  time_name_obj = soup.find('div', attrs={'class': 'flex items-center gap-2'})
  time = time_name_obj.time.text
  author = BeautifulSoup(str(time_name_obj), 'html.parser').findAll('a')[1].text

  return {'title': title, 'tags': tags_list, 'time': time, 'author': author}


In [147]:
def parse_data(n_pages=1, start_page=0):
  df = pd.DataFrame(columns=["title", "tags", "time", "author"])

  for i in range(start_page, n_pages):
    hrefs_list = get_hrefs_from_page(i)

    for post in hrefs_list:
      post_data = get_all_data_from_post(post)
      df = pd.concat([df, pd.DataFrame([post_data])], ignore_index=True)

      time.sleep(1)

    time.sleep(1)

  return df

In [150]:
df = parse_data(n_pages=10)

df.to_csv("ten_pages_result.csv")